In [146]:
# % load_ext autoreload
# % autoreload 2
import random
import numpy as np
import matplotlib.pyplot as plt
from utils import plot_NQueens, plot_evolution

# Local Search: Genetic Algorithm

## 1.Generating Phrases

In this section we consider a toy problem: generating a target phrases (e.g. "genetic algorithm") from an initial population of random strings. Assume the length of the target is known.

Problem Formulation: 

1. Direct Representation: just strings.
2. Exponential Search Space: $m^l$ where $m$ is the size of the alphabet (set of characters in our interest) and $l$ is the length of the string.

Now that we want to solve it with GA:
1. Chromosome: directly use the string itself.
2. Fitness: how match/close an individual is to the target.
3. Mating: combine two strings in someway to generate a new one.

The following cells will walk you through this problem to show the basic idea of GA

In [147]:
# setting up the problem
target = 'Genetic Algorithm'
u_case = [chr(x) for x in range(65, 91)]
l_case = [chr(x) for x in range(97, 123)]
gene_pool = u_case + l_case + [' ']  # all English characters and white spaces


def init_population(pop_size, gene_pool, state_length):
    """
    Randomly initialize a population for genetic algorithm
        pop_size  :  Number of individuals in population
        gene_pool   :  List of possible values for individuals
        state_length:  The length of each individual
    """

    population = []
    for _ in range(pop_size):
        new_individual = "".join(random.choices(gene_pool, k=state_length))
        population.append(new_individual)

    return population


# test
init_pop = init_population(10, gene_pool, len(target))
init_pop

['oLOmNKsSzMPkvqmcl',
 'OROihvWbvYzsQNmfP',
 'wi MncXscKSsRCYXr',
 'ldHjxrAIiOgnzfBCj',
 'rfJvxZrNyxnahBZuK',
 'gaRoHigDcktKNCtGQ',
 'rnDDWJMhZkxmHqFpy',
 'WpxPDlxKDjUZwhlNF',
 'jlJkGKcxjqvJUmcGf',
 'RRsRMBILkBSCHYTjF']

In [148]:
def fitness_fn(sample):
    fitness = 0
    length = len(target)
    for i in range(length):
        fitness += (abs(ord(target[i]) - ord(sample[i])))
    return fitness

# test
# fitness_fn(init_pop[0])

In [149]:
def select(r, population, fitness_fn):
    """
    TODO: select *r* samples from *population*
    the simplest choice is to sample from *population* with each individual weighted by its fitness
    """

    class Gene:
        def __init__(self, fitness, value):
            self.fitness = fitness
            self.value = value

        def __cmp__(self, other):
            if self.fitness > other.fitness:
                return -1
            elif self.fitness < other.fitness:
                return 1
            else:
                return 0

    genes = [Gene(fitness_fn(x), x) for x in population]
    genes = sorted(genes, key=lambda g: g.fitness)
    fitnesses = [g.fitness for g in genes]
    total_fitness = sum(fitnesses)
    select_pop = []
    length = len(fitnesses)
    for j in range(r):
        num = 0
        flag = random.randint(0, total_fitness)
        for i in range(length):
            num += fitnesses[i]
            if num >= flag:
                select_pop.append(genes[length - i - 1].value)
                break
    return select_pop


# test
parents = select(2, init_pop, fitness_fn)
parents

['jlJkGKcxjqvJUmcGf', 'WpxPDlxKDjUZwhlNF']

In [150]:
def recombine(x, y):
    """
    TODO: combine two parents to produce an offspring
    """
    length = len(x)
    flag = random.randint(0, length - 1)
    left = x[:flag]
    right = y[flag:]
    return left + right


def mutate(x, gene_pool, pmut):
    """
    apply mutation to *x* by randomly replacing one of its gene from *gene_pool*
    """
    if random.uniform(0, 1) >= pmut:
        return x

    n = len(x)
    g = len(gene_pool)
    c = random.randrange(0, n)
    r = random.randrange(0, g)

    new_gene = gene_pool[r]
    return x[:c] + new_gene + x[c + 1:]


# test
child = mutate(recombine(*parents), gene_pool, 0.1)
child

'jlJPDlxKDjUZwhlNF'

In [151]:
# now refactor things into a *Problem* abstraction
# you can directly reuse what you have implemented above
from abc import ABC, abstractmethod


class GAProblem(ABC):
    @abstractmethod
    def init_population(self, pop_size): pass

    @abstractmethod
    def fitness(self, sample): pass

    @abstractmethod
    def reproduce(self, population, mutation_rate): pass

    @abstractmethod
    def replacement(self, old, new): pass


class PhraseGeneration(GAProblem):
    def __init__(self, target, alphabet):
        self.target = target
        self.alphabet = alphabet

    def init_population(self, pop_size):
        # raise NotImplementedError()
        return init_population(pop_size, self.alphabet, len(self.target))

    def fitness(self, sample):
        fitness = 0
        length = len(self.target)
        for i in range(length):
            fitness += abs(ord(self.target[i]) - ord(sample[i]))
        return fitness

    def reproduce(self, population, mutation_rate):
        """
        TODO: generate the next generation of population

        hint: make a new individual with 
        
        mutate(recombine(*select(2, population, fitness_fn)), gene_pool, pmut)

        """
        return [mutate(recombine(*select(2, population, self.fitness)), self.alphabet, mutation_rate) for _ in
                range(len(population))]

    def replacement(self, old, new):
        """
        You can use your own strategy, for example retain some solutions from the old population
        """
        return new


def genetic_algorithm(
        problem: GAProblem,
        ngen, n_init_size, mutation_rate,
        log_interval=100
):
    population = problem.init_population(n_init_size)
    best = min(population, key=problem.fitness)
    history = [(0, list(map(problem.fitness, population)))]

    for gen in range(ngen):
        next_gen = problem.reproduce(population, mutation_rate)
        population = problem.replacement(population, next_gen)

        if gen % log_interval == 0:
            current_best = min(population, key=problem.fitness)
            if problem.fitness(current_best) < problem.fitness(best):
                best = current_best
            if best == 0:
                break
            print(f"Generation: {gen}/{ngen},\tBest: {best},\tFitness={problem.fitness(best)}")
            history.append((gen, list(map(problem.fitness, population))))

    history.append((ngen - 1, list(map(problem.fitness, population))))
    return best, history

In [152]:
# now set up the parameters
ngen = 1500
max_population = 120
mutation_rate = 0.08

sid = 12013006
target = f"Genetic Algorithm by {sid}"
u_case = [chr(x) for x in range(65, 91)]
l_case = [chr(x) for x in range(97, 123)]
digits = [str(i) for i in range(10)]
alphabet = u_case + l_case + digits + [' ']

problem = PhraseGeneration(target, alphabet)

solution, history = genetic_algorithm(problem, ngen, max_population, mutation_rate)
print(solution)

Generation: 0/1500,	Best: iTcTonag0fLXluwca2LzoBIJ4CRr4,	Fitness=580
Generation: 100/1500,	Best: MZrqgyQ4AeZksyujm5evM3LB54227,	Fitness=272
Generation: 200/1500,	Best: Mnrakym4Aeaksjujm5ev03AB54277,	Fitness=193
Generation: 300/1500,	Best: Jdrasdh4Aeeksjujm ez 32354277,	Fitness=85
Generation: 400/1500,	Best: Gdogsea Akeqsjujm az 12314257,	Fitness=35
Generation: 500/1500,	Best: Gdneska Alfosjujm bz 12114127,	Fitness=19
Generation: 600/1500,	Best: Gdnetia Alfosjuhm bz 12114127,	Fitness=14
Generation: 700/1500,	Best: Genetic Alforjuhm bz 12013116,	Fitness=6
Generation: 800/1500,	Best: Genetic Alforjuhm by 12013116,	Fitness=5
Generation: 900/1500,	Best: Genetic Algorithm by 12013106,	Fitness=1
Generation: 1000/1500,	Best: Genetic Algorithm by 12013006,	Fitness=0
Generation: 1100/1500,	Best: Genetic Algorithm by 12013006,	Fitness=0
Generation: 1200/1500,	Best: Genetic Algorithm by 12013006,	Fitness=0
Generation: 1300/1500,	Best: Genetic Algorithm by 12013006,	Fitness=0
Generation: 1400/1500,

## 2. N-Queens Problem

It is possible to solve the N-Queens problem with slight modifications.

For the problem:

1. Direct Representation: placement of queens.
2. Search Space: $n^n$, if we don't use any constraints.

To use GA:

Actually a new fitness function is the minimum modification you need for this problem.

Others are alomost the same as the previous problem.

In [ ]:
class NQueensProblem(GAProblem):
    def __init__(self, n):
        self.n = n
        self.max_fitness = n * (n - 1) // 2  # max number if non-attacking pairs

    def init_population(self, pop_size):
        # TODO:alomost the same as the previous problem.
        raise NotImplementedError()

    def fitness(self, queens):
        """
        TODO

        hint: count the non-attacking pairs
        """
        raise NotImplementedError()

    def reproduce(self, population, mutation_rate):
        # TODO:alomost the same as the previous problem.
        raise NotImplementedError()

    def replacement(self, old, new):
        """
        You can use your own strategy, for example retain some solutions from the old population
        """
        return new

    def __repr__(self):
        return f"{self.n}-Queens Problem"

In [ ]:
from utils import plot_NQueens

ngen = 1000
init_size = 120
mutation_rate = 0.08

n = 8
problem = NQueensProblem(n)
solution, history = genetic_algorithm(problem, ngen, init_size, mutation_rate)

In [ ]:
# Example of how to use this function
# plot_NQueens([4, 2, 0, 6, 1, 7, 5, 3])
# replace the parameter with your own results
plot_NQueens(solution)

In [ ]:
# Visualize the evolution of the polulation
bins = np.linspace(0, problem.max_fitness, problem.max_fitness)
plot_evolution(history, bins)